In [85]:
import pandas as pd
pd.set_option('display.max_colwidth',1000)
import numpy as np

### download news datasets for testing

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
import torch
# set a gpu device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# download a dataset from huggingface
from datasets import load_dataset
dataset1 = load_dataset('cnn_dailymail', '3.0.0', split='test')
dataset2 = load_dataset('xsum', '1.0.0', split='test')

/data/home/pelegdav/env/autoML/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /data/home/pelegdav/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


Using custom data configuration 1.0.0


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /data/home/pelegdav/.cache/huggingface/datasets/xsum/1.0.0/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


### Reduce dataset size by only picking out articles that contain organizations in the snp500

In [ ]:
# use spacy to extract the ticker symbols
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(text_to_summarize)
# iterate over doc and find all org entities
for ent in doc.ents:
    if ent.label_ == 'ORG':
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent", jupyter=True)


In [100]:
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")

# # Some text to summarize here
text_to_summarize = "National Commercial Bank (NCB), Saudi Arabia’s largest lender by assets, agreed to buy rival Samba Financial Group for $15 billion in the biggest banking takeover this year.NCB will pay 28.45 riyals ($7.58) for each Samba share, according to a statement on Sunday, valuing it at about 55.7 billion riyals. NCB will offer 0.739 new shares for each Samba share, at the lower end of the 0.736-0.787 ratio the banks set when they signed an initial framework agreement in June.The offer is a 3.5% premium to Samba’s Oct. 8 closing price of 27.50 riyals and about 24% higher than the level the shares traded at before the talks were made public. Bloomberg News first reported the merger discussions.The new bank will have total assets of more than $220 billion, creating the Gulf region’s third-largest lender. The entity’s $46 billion market capitalization nearly matches that of Qatar National Bank QPSC, which is still the Middle East’s biggest lender with about $268 billion of assets."
# dataset2_it = iter(dataset2)
# text_to_summarize = next(dataset2_it)['document']
print(text_to_summarize)
print()
# Tokenize our text
# If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
# input_ids = tokenizer(text_to_summarize, return_tensors="pt").input_ids.to(device)

# Generate the output (Here, we use beam search but you can also use any other strategy you like)
result = summarizer(text_to_summarize)[0]['summary_text']
# output = model.generate(
#     input_ids, 
#     max_length=64, 
#     num_beams=5, 
#     early_stopping=True
# )

# result = tokenizer.decode(output[0], skip_special_tokens=True)
# Finally, we can print the generated summary
print(result)
# Generated Output: Saudi bank to pay a 3.5% premium to Samba share price. Gulf region’s third-largest lender will have total assets of $220 billion


National Commercial Bank (NCB), Saudi Arabia’s largest lender by assets, agreed to buy rival Samba Financial Group for $15 billion in the biggest banking takeover this year.NCB will pay 28.45 riyals ($7.58) for each Samba share, according to a statement on Sunday, valuing it at about 55.7 billion riyals. NCB will offer 0.739 new shares for each Samba share, at the lower end of the 0.736-0.787 ratio the banks set when they signed an initial framework agreement in June.The offer is a 3.5% premium to Samba’s Oct. 8 closing price of 27.50 riyals and about 24% higher than the level the shares traded at before the talks were made public. Bloomberg News first reported the merger discussions.The new bank will have total assets of more than $220 billion, creating the Gulf region’s third-largest lender. The entity’s $46 billion market capitalization nearly matches that of Qatar National Bank QPSC, which is still the Middle East’s biggest lender with about $268 billion of assets.

Saudi Arabia's 

In [101]:
sentiment_to_int = {'positive': 1, 'negative': -1, 'neutral': 0}
int_to_sentiment = {1: 'positive', -1: 'negative', 0: 'neutral'}

def mean_sentiment(sentiment_list):
    # increase the sum of sentiment scores if the score is above 0.8
    sentiment_sum = np.sum([sentiment_to_int[sentiment['label']] if sentiment['score'] > 0.8 else 0 for sentiment in sentiment_list])
    return int_to_sentiment[sentiment_sum // len(sentiment_list)]

In [120]:
# split a string into a list of words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text_to_summarize)
paragraphs = [' '.join(tokens[i:i+64]) for i in range(0, len(tokens), 64)]

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
model = AutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
sentiments = [pipe(paragraph)[0] for paragraph in paragraphs]
# # tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
# # model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
# # pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
# # pipe = pipeline("sentiment-analysis", model='FinanceInc/auditor_sentiment_finetuned', tokenizer='FinanceInc/auditor_sentiment_finetuned')
print(mean_sentiment(sentiments))

neutral


### Extract ticker symbols from the results and append sentiment score of current article to the ticker's overall score

In [121]:
from spacy import displacy
from collections import defaultdict

orgs_tracked = defaultdict(list)

doc = nlp(result)
# iterate over doc and find all org entities
cur_orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
for org in cur_orgs:
    orgs_tracked[org].append(mean_sentiment(sentiments))
displacy.render(doc, style="ent", jupyter=True)

In [123]:
import requests

def get_ticker_symbol(company_name):
    # Define the API endpoint and the parameters for the request
    endpoint = "https://www.alphavantage.co/query"
    parameters = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "API_KEY",
    }
    # Send a GET request to the API endpoint
    response = requests.get(endpoint, params=parameters)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()
        # Check if the response contains information about the company
        print(data)
        if data["bestMatches"]:
            # Return the company name
            return data["bestMatches"][0]["1. symbol"]
    # If the request was unsuccessful, or the company information was not found, return None
    return None

company_name = "Microsoft Corporation"
ticker_symbol = get_ticker_symbol(company_name)
print(ticker_symbol)

{'bestMatches': [{'1. symbol': '0QYP.LON', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United Kingdom', '5. marketOpen': '08:00', '6. marketClose': '16:30', '7. timezone': 'UTC+01', '8. currency': 'USD', '9. matchScore': '1.0000'}, {'1. symbol': 'MSF.DEX', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'XETRA', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '1.0000'}, {'1. symbol': 'MSF.FRK', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'Frankfurt', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '1.0000'}, {'1. symbol': 'MSFT', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '1.0000'}, {'1. symbol': 'MSFT34.SAO', '2. name

In [119]:
ticker = get_ticker_symbol('National Commercial Bank')
print(ticker)

None
